#### ANN implementation on mnist dataset using python

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

We are directly reading the mnist dataset from the torchvsion's dataset module. This will return a dataset object which can directly be passed to DataLoader. It will however retrun a tensor of shape that replicates an image i.e 2-D

In [2]:
# Define a transformation to convert images to PyTorch tensors
transform = transforms.ToTensor()

# Load the Fashion MNIST training dataset
train_data = datasets.FashionMNIST(
    root='./data',  # Directory to store the dataset
    train=True,     # Specify this is the training set
    download=True,  # Download the dataset if not already present
    transform=transform
)

# Load the Fashion MNIST test dataset
test_data = datasets.FashionMNIST(
    root='./data',  # Directory to store the dataset
    train=False,    # Specify this is the test set
    download=True,  # Download the dataset if not already present
    transform=transform
)


Passing the fashion mnist dataset object into DataLoader.

In [3]:
train_dataloader = DataLoader(train_data, batch_size=256, shuffle=True, num_workers=0)
test_dataloader = DataLoader(test_data, batch_size=256, shuffle=False)


ANN model structure

In [4]:
class my_ann(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.network = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_features, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128,64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 10),
            nn.Softmax()
        )
    
    def forward(self, x):
        return self.network(x)


creating a model object and transfering the model to gpu.

In [5]:
num_features = train_data[0][0].view(-1).shape[0]
model = my_ann(num_features)

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# print(f"Using device: {device}")
# model.to(device)

Training the model

In [6]:
epochs = 100
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(epochs):
    for batch_features, batch_labels in train_dataloader:
        
        # batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        y_pred = model(batch_features)
        optimizer.zero_grad()
        loss = loss_fn(y_pred, batch_labels)
        loss.backward()
        optimizer.step()

        torch.mps.empty_cache()
    
    if epoch%10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")
    


Epoch 1/100, Loss: 1.5954
Epoch 11/100, Loss: 1.6087
Epoch 21/100, Loss: 1.7088
Epoch 31/100, Loss: 1.6309
Epoch 41/100, Loss: 1.6562
Epoch 51/100, Loss: 1.5917
Epoch 61/100, Loss: 1.5574
Epoch 71/100, Loss: 1.6582
Epoch 81/100, Loss: 1.5983
Epoch 91/100, Loss: 1.5859


Model evaluation

In [7]:
model.eval()

accuracy_list = []
for batch_features, batch_labels in test_dataloader:
    # batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
    y_pred = model(batch_features)
    _, predicted = torch.max(y_pred, 1)
    accuracy = accuracy_score(batch_labels.numpy(), predicted.numpy())
    accuracy_list.append(accuracy)
average_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f"Average Test Accuracy: {average_accuracy:.4f}")
    


Average Test Accuracy: 0.8236


In [8]:
# model.eval()

accuracy_list = []
for batch_features, batch_labels in train_dataloader:
    # batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
    y_pred = model(batch_features)
    _, predicted = torch.max(y_pred, 1)
    accuracy = accuracy_score(batch_labels.numpy(), predicted.numpy())
    accuracy_list.append(accuracy)
average_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f"Average Train Accuracy: {average_accuracy:.4f}")
    


Average Train Accuracy: 0.8338
